## Assignment 1
### Senan Gaffori, 20949022
### Ayush Bhargava, 20889700

### 1.1 - An optimization problem with Gurobipy

In [7]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model("DemoExample")

x = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x")
y = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name="y")

m.setObjective(x + y, GRB.MAXIMIZE)

m.addConstr(x+2*y <=4, "c1")
m.addConstr(x*4+3*y <= 12, "c2")

m.optimize()

if m.status == GRB.OPTIMAL:
    if m.status == GRB.OPTIMAL:
        print(f"Objective Value: {m.objVal}")
        print(f"x: {x.X}, y: {y.X}")

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x9a331afa
Coefficient statistics:
  Matrix range     [1e+00, 4e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 1e+01]
Presolve time: 0.01s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.0000000e+30   3.250000e+30   2.000000e+00      0s
       2    3.2000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  3.200000000e+00
Objective Value: 3.2
x: 2.4, y: 0.8


### 1.2 - An optimization problem with CPLEX

In [ ]:
%pip install docplex

from docplex.mp.model import Model

# Create a model
mdl = Model(name='DemoDocplex')

# Variables
x = mdl.continuous_var(name='x', lb=0)
y = mdl.continuous_var(name='y', lb=0)

# Objective: Maximize x + y
mdl.maximize(x + y)

# Constraints
mdl.add_constraint(x + 2*y <= 4, 'c1')
mdl.add_constraint(x*4 + 3*y <= 12, 'c2')

# Solve
sol = mdl.solve(log_output=True)

# Output
if sol:
    print(f"Objective Value: {mdl.objective_value}")
    mdl.print_solution()
else:
    print("No solution found")

From 1.1 and 1.2 we see the final solution has optimal values of:
x = 2.4
y = 0.8
z = 3.2

## 2 - Lab Assignment

## Problem Formulation

## CFLP - Sets, Parameters and Decsion Variables

### Sets:

$I$: Set of facilities ($f_1, f_2, f_3, f_4, f_5, f_6, f_7$)
<br>
$J$: Set of customers ($c_1, c_2, c_3, c_4, c_5, c_6, c_7, c_8, c_9, c_{10}$)

### Parameters:
$f_i$: Fixed cost of opening facility $i \in I$
<br>
$u_i$: Capacity of facility $i \in I$
<br>
$d_j$: Demand of customer $j \in J$
<br>
$c_{ij}$: Distance of shipping from facility $i$ to customer $j$

### Decision Variables:
$y_i \in \{0,1\}$: Binary variable indicating if facility $i$ is open
<br>
$x_{ij} \geq 0$: Amount shipped from facility $i$ to customer $j$

## Constraints

### Demand Satisfaction - Each Customer Demand Must Be Met
$$
\sum_{i \in I} x_{ij} = d_j \quad \forall j \in J
$$

### Facility Capacity - A facility can only ship up to its capacity if it is open:
$$
\sum_{j \in J} x_{ij} \leq u_i y_i \quad \forall i \in I
$$

### Variable Domains
$$
x_{ij} \geq 0 \quad \forall i \in I, j \in J
$$

$$
y_i \in \{0, 1\} \quad \forall i \in I
$$



# Code Implementation w/ Gurobi

## Import Libaries and Read CSV file w/ Customer Data

In [9]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# --- Load CSV ---
df = pd.read_csv("LabData.csv")

# --- Extract facility and customer data ---
facility_data = df[['Facility', 'Fixed Cost ($)', 'Capacity (units)']].dropna().reset_index(drop=True)
facilities = facility_data['Facility'].tolist()
fixed_cost = dict(zip(facility_data['Facility'], facility_data['Fixed Cost ($)']))
capacity = dict(zip(facility_data['Facility'], facility_data['Capacity (units)']))

customer_cols = df.columns[df.columns.get_loc('Facility.1')+1:]
customers = customer_cols.tolist()

customer_data = df[['Customer', 'Demand (units)']].dropna().reset_index(drop=True)
demand = dict(zip(customer_data['Customer'], customer_data['Demand (units)']))

distance = {}
for i, row in facility_data.iterrows():
    facility = row['Facility']
    for customer in customers:
        distance[(facility, customer)] = df.loc[i, customer]

## Build Model

In [10]:
# --- Gurobi Model ---
m = Model("CapacitatedFacilityLocation")

y = m.addVars(facilities, vtype=GRB.BINARY, name="Open")
x = m.addVars(facilities, customers, vtype=GRB.CONTINUOUS, name="Ship")

m.setObjective(
    quicksum(fixed_cost[i] * y[i] for i in facilities) +
    quicksum(distance[i, j] * x[i, j] for i in facilities for j in customers),
    GRB.MINIMIZE
)

for j in customers:
    m.addConstr(quicksum(x[i, j] for i in facilities) == demand[j], name=f"Demand_{j}")

for i in facilities:
    m.addConstr(quicksum(x[i, j] for j in customers) <= capacity[i] * y[i], name=f"Capacity_{i}")

m.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 17 rows, 77 columns and 147 nonzeros
Model fingerprint: 0xee20544e
Variable types: 70 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+01]
  Objective range  [8e+00, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 3e+01]
Presolve time: 0.02s
Presolved: 17 rows, 77 columns, 147 nonzeros
Variable types: 70 continuous, 7 integer (7 binary)
Found heuristic solution: objective 98480.000000

Root relaxation: objective 5.489500e+04, 49 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 54895.0000    0    1 98480.0000 54895.0000  44.3%     -    0s
H 

## Print Solution

In [11]:
# --- Print results ---
if m.status == GRB.OPTIMAL:
    print(f"Total Cost: {m.objVal:.2f}")
    for i in facilities:
        if y[i].x > 0.5:
            print(f"\nFacility {i} is OPEN")
            for j in customers:
                if x[i, j].x > 0:
                    print(f"  Ships {x[i, j].x} units to Customer {j}")
else:
    print("No optimal solution found.")


Total Cost: 56535.00

Facility F1 is OPEN
  Ships 5.0 units to Customer C2
  Ships 27.0 units to Customer C5
  Ships 8.0 units to Customer C10

Facility F2 is OPEN
  Ships 18.0 units to Customer C1
  Ships 25.0 units to Customer C3
  Ships 17.0 units to Customer C8

Facility F5 is OPEN
  Ships 30.0 units to Customer C4
  Ships 20.0 units to Customer C6
  Ships 2.0 units to Customer C8
  Ships 13.0 units to Customer C10

Facility F6 is OPEN
  Ships 17.0 units to Customer C2
  Ships 15.0 units to Customer C7
  Ships 23.0 units to Customer C9


## Final Solutions


**Total Cost:** $56535.00$

**Facility F1 is OPEN**
* Ships $5.0$ units to Customer C2
* Ships $27.0$ units to Customer C5
* Ships $8.0$ units to Customer C10

**Facility F2 is OPEN**
* Ships $18.0$ units to Customer C1
* Ships $25.0$ units to Customer C3
* Ships $17.0$ units to Customer C8

**Facility F5 is OPEN**
* Ships $30.0$ units to Customer C4
* Ships $20.0$ units to Customer C6
* Ships $2.0$ units to Customer C8
* Ships $13.0$ units to Customer C10

**Facility F6 is OPEN**
* Ships $17.0$ units to Customer C2
* Ships $15.0$ units to Customer C7
* Ships $23.0$ units to Customer C9

## GenAI Disclosure

GenAI and LLMs were used to format the text in the jupyter notebook to be a user friendly reading form.
